# Deep Neural Decision Tree

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

In [ ]:
#Decision Nodes
def create_decision_node(model_output, threshold=0.5):
    decisions = np.where(model_output > threshold, 1, 0)
    return decisions

#Train the Model
def train_model(model, x_train, y_train):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=100, batch_size=10, verbose=0)

# Prediction
def predict(model, x_test):
    model_output = model.predict(x_test)
    decisions = create_decision_node(model_output)
    return decisions

In [ ]:
#IRIS dataset

# Load the Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
iris = pd.read_csv(url, names=columns)

# Preprocess
X = iris.drop('Species', axis=1)
y = iris['Species']

# Encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])

train_model(model, X_train, y_train)

#Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 136ms/step - loss: 0.0698 - accuracy: 0.9667
Accuracy: 96.67%


In [ ]:
#Haberman’s Survival dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data"
column = ["age", "year_of_treatment", "positive_lymph_nodes", "survival_status_after_5_years"]
haberman = pd.read_csv(url, names=column)

# Preprocess
X = haberman.drop("survival_status_after_5_years", axis=1)
y = haberman["survival_status_after_5_years"]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(3,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
train_model(model, X_train, y_train)

#Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.7097
Accuracy: 70.97%


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


In [ ]:
#Car Evaluation dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
column = ["buying_price", "maintenance_cost", "number_of_doors", "number_of_persons", "lug_boot", "safety", "decision"]
car = pd.read_csv(url, names=column)

# Preprocess
X = car.drop("decision", axis=1)
y = car["decision"]

# Encode
one_hot_encoder = OneHotEncoder()
X_encoded = one_hot_encoder.fit_transform(X).toarray()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2)

# Encode
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(21,)),
        Dense(32, activation='relu'),
        Dense(4, activation='softmax')
    ])

train_model(model, X_train, y_train_encoded)

#Evaluate
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Accuracy: {accuracy * 100:.2f}%")

11/11 [==============================] - 0s 3ms/step - loss: 1.0170e-04 - accuracy: 1.0000
Accuracy: 100.00%


In [ ]:
# Breast Cancer Wisconsin
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
column = ["Sample_code_number", "Clump_thickness", "Uniformity_of_cell_size", "Uniformity_of_cell_shape", "Marginal_adhesion", "Single_epithelial_cell_size	", "Bare_nuclei", "Bland_chromatin","Normal_nucleoli", "Mitoses", "Class"]
cancer = pd.read_csv(url, names=column)

# Preprocess
cancer.replace("?", np.nan, inplace=True)
cancer.dropna(inplace=True)

X = cancer.drop("Class", axis=1)
y = cancer["Class"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

y_encoded = to_categorical(y_encoded)

#Split
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X_scaled, y_encoded, test_size=0.2)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(y_encoded.shape[1], activation='softmax')
    ])
train_model(model, X_train, y_train_encoded)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Accuracy: {accuracy * 100:.2f}%")

5/5 [==============================] - 0s 3ms/step - loss: 0.2758 - accuracy: 0.9635
Accuracy: 96.35%


In [ ]:
#Poker Hands
url_train = "https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data"
url_test = "https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data"
column = ["S1", "C1", "S2", "C2", "S3", "C3", "S4", "C4", "S5", "C5", "Class"]
poker_train = pd.read_csv(url_train, names=column)
poker_est = pd.read_csv(url_test, names=column)

#Train Test
X_train = poker_train.drop("Class", axis=1)
y_train = poker_train["Class"]
X_test = poker_test.drop("Class", axis=1)
y_test = poker_test["Class"]

#Encode
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

one_hot_encoder = OneHotEncoder(categories='auto')
y_train_encoded = one_hot_encoder.fit_transform(np.array(y_train_encoded).reshape(-1, 1)).toarray()
y_test_encoded = one_hot_encoder.transform(np.array(y_test_encoded).reshape(-1, 1)).toarray()

num_classes = y_train_encoded.shape[1]

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
train_model(model, X_train, y_train_encoded)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Accuracy: {accuracy * 100:.2f}%")

31250/31250 [==============================] - 73s 2ms/step - loss: 0.0934 - accuracy: 0.9813
Accuracy: 98.13%


In [ ]:
#German Credit Dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
column = ["Status of existing checking account", "Duration", "Credit history", "Purpose", "Credit amount", "Savings account/bonds", "Present employment since", "Installment rate in percentage of disposable income", "Personal status and sex", "Other debtors / guarantors", "Present residence since", "Property", "Age", "Other installment plans", "Housing", "Number of existing credits at this bank", "Telephone", "Foreign worker", "Credit risk"]
credit = pd.read_csv(url, names=column, sep=' ', header=None)

# Identify categorical and numerical columns
categorical_cols = ["Status of existing checking account", "Credit history", "Purpose", "Savings account/bonds","Present employment since", "Personal status and sex", "Other debtors / guarantors", "Property", "Other installment plans", "Housing", "Job", "Telephone", "Foreign worker"]
numerical_cols = ["Duration", "Credit amount", "Installment rate in percentage of disposable income","Present residence since", "Age", "Number of existing credits at this bank", "Number of people being liable to provide maintenance for"]

# Preprocess the data
X = credit.drop("Credit risk", axis=1)
y = credit["Credit risk"]

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Create a column transformer for preprocessing
preprocessor = make_column_transformer(
    (numerical_transformer, numerical_cols),
    (categorical_transformer, categorical_cols)
)

X_processed = preprocessor.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2)

model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
# Train the model and evaluate its accuracy
train_model(model, X_train, y_train_encoded)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Accuracy: {accuracy * 100:.2f}%")

7/7 [==============================] - 0s 4ms/step - loss: 2.5119 - accuracy: 0.5450
Accuracy: 54.50%



# **Experiment with different hyperparameters**

**Number of Batch-Size**

In [ ]:
### LARGE NUMBER OF Batch Size
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

def create_decision_node(model_output, threshold=0.5):
    decisions = np.where(model_output > threshold, 1, 0)
    return decisions

#Train the Model
def train_model(model, x_train, y_train):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=100, batch_size=50, verbose=0)

# Prediction
def predict(model, x_test):
    model_output = model.predict(x_test)
    decisions = create_decision_node(model_output)
    return decisions


#IRIS dataset

# Load the Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
iris = pd.read_csv(url, names=columns)

# Preprocess
X = iris.drop('Species', axis=1)
y = iris['Species']

# Encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])

train_model(model, X_train, y_train)

#Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 124ms/step - loss: 0.1219 - accuracy: 0.9667
Accuracy: 96.67%


In [ ]:
### SMALL NUMBER OF Batch Size
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

def create_decision_node(model_output, threshold=0.5):
    decisions = np.where(model_output > threshold, 1, 0)
    return decisions

#Train the Model
def train_model(model, x_train, y_train):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=100, batch_size=5, verbose=0)

# Prediction
def predict(model, x_test):
    model_output = model.predict(x_test)
    decisions = create_decision_node(model_output)
    return decisions


#IRIS dataset

# Load the Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
iris = pd.read_csv(url, names=columns)

# Preprocess
X = iris.drop('Species', axis=1)
y = iris['Species']

# Encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])

train_model(model, X_train, y_train)

#Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 133ms/step - loss: 0.0631 - accuracy: 1.0000
Accuracy: 100.00%


**Number of Epochs**

In [ ]:
### SMALL NUMBER OF Epochs
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

def create_decision_node(model_output, threshold=0.5):
    decisions = np.where(model_output > threshold, 1, 0)
    return decisions

#Train the Model
def train_model(model, x_train, y_train):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5, batch_size=10, verbose=0)

# Prediction
def predict(model, x_test):
    model_output = model.predict(x_test)
    decisions = create_decision_node(model_output)
    return decisions


#IRIS dataset

# Load the Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
iris = pd.read_csv(url, names=columns)

# Preprocess
X = iris.drop('Species', axis=1)
y = iris['Species']

# Encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])

train_model(model, X_train, y_train)

#Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 186ms/step - loss: 0.5256 - accuracy: 0.8333
Accuracy: 83.33%


In [ ]:
### SMALL NUMBER OF Epochs
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

def create_decision_node(model_output, threshold=0.5):
    decisions = np.where(model_output > threshold, 1, 0)
    return decisions

#Train the Model
def train_model(model, x_train, y_train):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=8, batch_size=10, verbose=0)

# Prediction
def predict(model, x_test):
    model_output = model.predict(x_test)
    decisions = create_decision_node(model_output)
    return decisions


#IRIS dataset

# Load the Iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
iris = pd.read_csv(url, names=columns)

# Preprocess
X = iris.drop('Species', axis=1)
y = iris['Species']

# Encode
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])

train_model(model, X_train, y_train)

#Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 137ms/step - loss: 0.5425 - accuracy: 1.0000
Accuracy: 100.00%
